In [12]:
import pandas as pd

In [13]:
df = pd.read_csv('venda_aluguel.txt', sep='\t')
df.columns = ['acao', 'pessoa1', 'pessoa2', 'parcela', 'objeto', 'tipo_preco', 'rodada', 'jogo']
#'saldo_CD1', 'saldo_CD2', 'saldo_CD3', 'saldo_AT1', 'saldo_AT2', 'saldo_AT3', 'saldo_EmpAgr', 'saldo_EmpFer', 'saldo_EmpMaq', 'saldo_EmpSem', 'saldo_FiscalAT', 'saldo_FiscalCD', 'saldo_PrefeitoAT', 'saldo_PrefeitoCD', 'poluicao_mundial', 'selo', 'prox_taxa', 'val_taxa', 'tratamento'

In [14]:
#Inclui colunas para normalizar o banco
serie = pd.Series([])
df['saldo_CD1'] = serie
df['saldo_CD2'] = serie
df['saldo_CD3'] = serie
df['saldo_AT1'] = serie
df['saldo_AT2'] = serie
df['saldo_AT3'] = serie
df['saldo_EmpAgr'] = serie
df['saldo_EmpFer'] = serie
df['saldo_EmpMaq'] = serie
df['saldo_EmpSem'] = serie
df['saldo_FiscalAT'] = serie
df['saldo_FiscalCD'] = serie
df['saldo_PrefeitoAT'] = serie
df['saldo_PrefeitoCD'] = serie
df['poluicao_mundial'] = serie
df['selo'] = serie
df['prox_taxa'] = serie
df['val_taxa'] = serie
df['tratamento'] = serie

In [15]:
#Carrego o restante dos dados
df_acoes = pd.read_csv('acao_ambiental.txt', sep='\t')
df_imposto = pd.read_csv('imposto.txt', sep='\t')
df_multa = pd.read_csv('multa.txt', sep='\t')
df_recebe_selo = pd.read_csv('recebe_selo.txt', sep='\t')
df_transferencia = pd.read_csv('transferencia.txt', sep='\t')
df_troca_taxa = pd.read_csv('troca_taxa.txt', sep='\t')
df_taxa_vigente = pd.read_csv('taxa_vigente.txt', sep='\t')

In [17]:
#Separar venda do aluguel

df['acao'] = df['pessoa1'].apply(lambda x: 'aluguel' if x == 'EmpMaq' else'venda')

In [18]:
df['tipo_preco2'] = df['tipo_preco'].apply(lambda x: '0' if x == 'normal' else ('-5' if x == 'baixo' else ('+5' if x == 'alto' else x)))

In [19]:
df = pd.concat([df, df_acoes, df_imposto, df_multa, df_recebe_selo, df_transferencia, df_troca_taxa])

In [20]:
df.drop(['valor'], axis=1)

,acao,pessoa1,pessoa2,parcela,objeto,tipo_preco,rodada,jogo,saldo_CD1,saldo_CD2,...,saldo_PrefeitoAT,saldo_PrefeitoCD,poluicao_mundial,selo,prox_taxa,val_taxa,tratamento,tipo_preco2,valor3,valor2
0,venda,EmpSem,AT1,1.0,Soja,baixo,1,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5,NaN,NaN
1,venda,EmpFer,AT1,1.0,Super premium,baixo,1,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5,NaN,NaN
2,aluguel,EmpMaq,AT1,1.0,Maquinas 3,baixo,1,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5,NaN,NaN
3,venda,EmpSem,AT1,2.0,Soja,baixo,1,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5,NaN,NaN
4,venda,EmpFer,AT1,2.0,Super premium,baixo,1,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,nova taxa,CD2,NaN,NaN,NaN,NaN,4,3,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,15.0%,NaN,NaN,NaN,NaN
22,nova taxa,CD2,NaN,NaN,NaN,NaN,4,3,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,25.0%,NaN,NaN,NaN,NaN
23,nova taxa,AT2,NaN,NaN,NaN,NaN,1,2,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,5.0,NaN,NaN,NaN,NaN
24,nova taxa,AT2,NaN,NaN,NaN,NaN,1,2,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,35.0%,NaN,NaN,NaN,NaN


In [21]:
jogos = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
valores = [3, 5, 4]

In [22]:
a = 0
for i in range(2,5):
    for y in range(1, (valores[a]+1)):
        jogos[a] = pd.concat([jogos[a],df[(df['jogo'] == i)][df[(df['jogo']== i)]['rodada'] == y]])
    a+=1

In [24]:
sem_anterior = ['', 0, 0]
fer_anterior = ''
produtividade = [] # produtividade dos personagens na rodada AT1 AT2 AT3 CD1 CD2 CD3 EmpAgr EmpFer EmpMaq EmpSem
taxa = []
produtividade_agricultor = 0
impostos = []
def calcula_acao(acao):
    rodada = jogos[0].iloc[[x]]['rodada'].tolist()[0]
    jogo = jogos[0].iloc[[x]]['jogo'].tolist()[0]
    if acao == 'venda':
        produto = jogos[0].iloc[[x]]['objeto'].tolist()[0]
        empresario = jogos[0].iloc[[x]]['pessoa1'].tolist()[0]
        agricultor = jogos[0].iloc[[x]]['pessoa2'].tolist()[0]
        tipo_preco = int(jogos[0].iloc[[x]]['tipo_preco2'].tolist()[0])  
        s_pessoa1 = 0
        s_pessoa2 = 0
        poluicao = 0
        
        if empresario == 'EmpSem':
            fer_anterior = ''
            sem_anterior[2] = 0
            sem_anterior[1] = 0
            sem_anterior[0] = produto
            if produto == 'Soja':
                s_pessoa1 = 30+tipo_preco
                s_pessoa2 = -(30+tipo_preco)+10
                poluicao = 30+3
                sem_anterior[2] = 30
            elif produto == 'Arroz':
                s_pessoa1 = 20+tipo_preco
                s_pessoa2 = -(20+tipo_preco)+10
                poluicao = 20+2
                sem_anterior[2] = 20
            else:
                #Só pode ser hortaliças!!
                s_pessoa1 = 10+tipo_preco
                s_pessoa2 = -(10+tipo_preco)+10
                poluicao = 10+1
                sem_anterior[2] = 10
            sem_anterior[1] += 10
            produtividade[9] += s_pessoa1 # produtividade do empresario de sementes
            produtividade_agricultor = 10
                
        elif empresario == 'EmpFer':
            if produto == 'Comum':
                s_pessoa1 = 10+tipo_preco
                s_pessoa2 = -(10+tipo_preco)+20
                sem_anterior[1] += 20
                fer_anterior = 'Comum'
                poluicao = 9
                produtividade_agricultor = 20
            elif produto == 'Premium':
                s_pessoa1 = 20+tipo_preco
                s_pessoa2 = -(20+tipo_preco)+30
                sem_anterior[1] += 30
                fer_anterior = 'Premium'
                poluicao = 6
                produtividade_agricultor = 30
            else:
                #Só pode ser o super premium!!
                s_pessoa1 = 30+tipo_preco
                s_pessoa2 = - (30+tipo_preco)+40
                sem_anterior[1] += 40
                fer_anterior = 'Super premium'
                poluicao = 3
                produtividade_agricultor = 40
            produtividade[7] += s_pessoa1 # produtividade do empresario de fertilizante
        
        else:
            #Só pode ser o empresario de agrotóxico!!
            if produto == 'Comum':
                s_pessoa1 = 10+tipo_preco
                s_pessoa2 = - (10+tipo_preco)
                if fer_anterior != '':
                    if fer_anterior == 'Comum':
                        s_pessoa2 += 40
                        produtividade_agricultor = 40
                        sem_anterior[1] += 40
                    elif fer_anterior == 'Premium':
                        s_pessoa2 += 60
                        produtividade_agricultor = 60
                        sem_anterior[1] += 60
                    else:
                        s_pessoa2 += 80
                        produtividade_agricultor = 80
                        sem_anterior[1] += 80
                else:
                    s_pessoa2 += 20
                    produtividade_agricultor = 20
                if sem_anterior[0] == 'Hortalica':
                    poluicao = 20
                    sem_anterior[2] += 20
                elif sem_anterior[0] == 'Arroz':
                    poluicao = 40
                    sem_anterior[2] += 40
                else: 
                    poluicao = 60                    
                    sem_anterior[2] += 60
                    sem_anterior[1] = sem_anterior[1]*2
                poluicao += 3
            elif produto == 'Premium':
                s_pessoa1 = 20+tipo_preco
                s_pessoa2 = - (20+tipo_preco)
                if fer_anterior != '':
                    if fer_anterior == 'Comum':
                        s_pessoa2 += 100
                        produtividade_agricultor = 100
                        sem_anterior[1] += 100
                    elif fer_anterior == 'Premium':
                        s_pessoa2 += 150
                        produtividade_agricultor = 150
                        sem_anterior[1] += 150
                    else:
                        s_pessoa2 += 200
                        produtividade_agricultor = 200
                        sem_anterior[1] += 200
                else:
                    s_pessoa2 += 50
                    produtividade_agricultor = 50
                if sem_anterior[0] == 'Hortalica':
                    poluicao = 50
                    sem_anterior[2] += 50
                elif sem_anterior[0] == 'Arroz':
                    poluicao = 100
                    sem_anterior[2] += 100
                else: 
                    poluicao = 150
                    sem_anterior[2] += 150
                    sem_anterior[1] = sem_anterior[1]*2
                poluicao += 2
            else:
                #Só pode ser o super premium!!
                s_pessoa1 = 30+tipo_preco
                s_pessoa2 = - (30+tipo_preco)
                if fer_anterior != '':
                    if fer_anterior == 'Comum':
                        s_pessoa2 += 180
                        produtividade_agricultor = 180
                        sem_anterior[1] += 180
                    elif fer_anterior == 'Premium':
                        s_pessoa2 += 270
                        produtividade_agricultor = 270
                        sem_anterior[1] += 270
                    else:
                        s_pessoa2 += 360
                        produtividade_agricultor = 360
                        sem_anterior[1] += 360
                else:
                    s_pessoa2 += 90
                    produtividade_agricultor = 90
                if sem_anterior[0] == 'Hortalica':
                    poluicao = 90
                    sem_anterior[2] += 90
                elif sem_anterior[0] == 'Arroz':
                    poluicao = 180
                    sem_anterior[2] += 180
                else: 
                    poluicao = 270
                    sem_anterior[2] += 270
                    sem_anterior[1] = sem_anterior[1]*2
                poluicao += 1
            s_pessoa2 += sem_anterior[1]
            produtividade[6] += s_pessoa1 # produtividade do empresario de agrotoxico
            
    elif acao == "aluguel":
        s_pessoa1 = 0
        s_pessoa2 = 0
        produto = jogos[0].iloc[[x]]['objeto'].tolist()[0]
        agricultor = jogos[0].iloc[[x]]['pessoa2'].tolist()[0]
        tipo_preco = int(jogos[0].iloc[[x]]['tipo_preco2'].tolist()[0]) 
        
        if produto == 'Maquinas 1':
            s_pessoa1 = 30+tipo_preco
            s_pessoa2 = -(30+tipo_preco)
            poluicao = 3
            if fer_anterior != '':
                if fer_anterior == 'Comum':
                    sem_anterior[1] += 40 
                    s_pessoa2 += 40
                    produtividade_agricultor = 40
                elif fer_anterior == 'Premium':
                    sem_anterior[1] += 60
                    s_pessoa2 += 60
                    produtividade_agricultor = 60
                else:
                    sem_anterior[1] += 80
                    s_pessoa2 += 80            
                    produtividade_agricultor = 80
        elif produto == 'Maquinas 2':
            s_pessoa1 = 60+tipo_preco
            s_pessoa2 = -(60+tipo_preco)
            poluicao = 6
            if fer_anterior != '':
                if fer_anterior == 'Comum':
                    sem_anterior[1] += 100 
                    s_pessoa2 += 100
                    produtividade_agricultor = 100
                elif fer_anterior == 'Premium':
                    sem_anterior[1] += 150
                    s_pessoa2 += 150
                    produtividade_agricultor = 150
                else:
                    sem_anterior[1] += 200
                    s_pessoa2 += 200
                    produtividade_agricultor = 200
        elif produto == 'Maquinas 3':
            s_pessoa1 = 90+tipo_preco
            s_pessoa2 = -(90+tipo_preco)
            poluicao = 9
            if fer_anterior != '':
                if fer_anterior == 'Comum':
                    sem_anterior[1] += 180 
                    s_pessoa2 += 180
                    produtividade_agricultor = 180
                elif fer_anterior == 'Premium':
                    sem_anterior[1] += 270
                    s_pessoa2 += 270
                    produtividade_agricultor = 270
                else:
                    sem_anterior[1] += 360
                    s_pessoa2 += 360
                    produtividade_agricultor = 360
        else:
            #É o pulverizador
            s_pessoa1 = 400+tipo_preco
            s_pessoa2 = -(400+tipo_preco)
            poluicao = -sem_anterior[2]/2
            poluicao += 40
        produtividade[8] += s_pessoa1 # produtividade do empresario de maquinas
    elif acao == "imposto": 
        prefeito = jogos[0].iloc[[x]]['pessoa1'].tolist()[0]
        produtor = jogos[0].iloc[[x]]['pessoa2'].tolist()[0] # empresario ou agricultor
        atlantis = ['AT1', 'AT2', 'AT3', 'EmpMaq', 'EmpFer']
        cidadela = ['CD1', 'CD2', 'CD3', 'EmpSem', 'EmpAgr']
        if produtor in atlantis:
            cidade = 'Atlantis'
            if agricultor == 'AT1':
                producao = produtividade[0]
            elif agricultor == 'AT2':
                producao = produtividade[1]
            elif agricultor == 'AT3':
                producao = produtividade[2]
            elif agricultor == 'EmpMaq':
                producao = produtividade[8]
            else:
                 producao = produtividade[6]                          
        else:
            cidade = 'Cidadela'
            if agricultor == 'CD1':
                producao = produtividade[3]
            elif agricultor == 'CD2':
                producao = produtividade[4]
            elif agricultor == 'CD3':
                producao = produtividade[5]
            elif agricultor == 'EmpSem':
                producao = produtividade[9]
            else:
                producao = produtividade[7]        
        if producao == 0:  # baixa produtividada
            tipo_taxa = 1
        elif producao > 0 and producao <= 200: # media produtividada
            tipo_taxa = 2
        else: # alta produtividada
            tipo_taxa = 3
        valor = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['tipo_taxa'] == tipo_taxa)][(df_taxa_vigente['cidade'] == cidade)].iloc[:]['valor'].tolist()[0]
        if '%' in valor:
            valor = valor.replace('%','')
            valor = int(valor)/100
            imposto = valor * producao
        else:
            imposto = int(valor)
        impostos.append(imposto) # para utilizar no desconto de quem conseguiu selo verde
        s_pessoa1 = jogos[0].iloc[[x]]['saldo_'+pessoa1].tolist()[0] # arrumar para pegar da linha acima mais proxima
        s_pessoa2 = jogos[0].iloc[[x]]['saldo_'+pessoa2].tolist()[0]
        s_pessoa1 = s_pessoa1 + imposto 
        s_pessoa2 = s_pessoa2 - imposto 
        produtividade.clear() # limpa a lista para a proxima rodada
        produtividade_agricultor = 0

    elif acao == "multa":
        pessoa2 = jogos[0].iloc[[x]]['pessoa2'].tolist()[0]
        s_pessoa2 = jogos[0].iloc[[x]]['saldo_'+pessoa2].tolist()[0]
        valor = jogos[0].iloc[[x]]['valor2'].tolist()[0]
        s_pessoa2 = s_pessoa2 - valor   

    elif acao == "selo": # 5% de desconto no imposto por parcela, o dinheiro descontado é o do caixa da prefeitura
        pessoa1 = jogos[0].iloc[[x]]['pessoa1'].tolist()[0] # fiscal
        pessoa2 = jogos[0].iloc[[x]]['pessoa2'].tolist()[0] # agricultor 
        prefeito_atlantis = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Atlantis')].iloc[:]['personagem'].tolist()[0]     
        prefeito_cidadela = df_taxa_vigente[(df_taxa_vigente['jogo'] == jogo)][(df_taxa_vigente['rodada'] == rodada)][(df_taxa_vigente['cidade'] == 'Cidadela')].iloc[:]['personagem'].tolist()[0]     
        atlantis = ['AT1', 'AT2', 'AT3', 'EmpMaq', 'EmpFer']
        cidadela = ['CD1', 'CD2', 'CD3', 'EmpSem', 'EmpAgr']
        if pessoa2 in atlantis:
            s_pessoa1 = jogos[0].iloc[[x]]['saldo_'+prefeito_atlantis].tolist()[0] 
        else:
            s_pessoa1 = jogos[0].iloc[[x]]['saldo_'+prefeito_cidadela].tolist()[0]
        s_pessoa2 = jogos[0].iloc[[x]]['saldo_'+pessoa2].tolist()[0]
        if pessoa2 == 'AT1':
            desconto = impostos[4]*0.05
        elif pessoa2 == 'AT2':
            desconto = impostos[6]*0.05
        elif pessoa2 == 'AT3':
            desconto = impostos[8]*0.05
        elif pessoa2 == 'CD1':
            desconto = impostos[5]*0.05
        elif pessoa2 == 'CD2':
            desconto = impostos[7]*0.05
        elif pessoa2 == 'CD3':
            desconto = impostos[9]*0.05    
        s_pessoa1 = s_pessoa1 - desconto # prefeito
        s_pessoa2 = s_pessoa2 + desconto # agricultor
        impostos.clear()
    elif acao == "transferencia":
        pessoa1 = jogos[0].iloc[[x]]['pessoa1'].tolist()[0]
        pessoa2 = jogos[0].iloc[[x]]['pessoa2'].tolist()[0]
        s_pessoa1 = jogos[0].iloc[[x]]['saldo_'+pessoa1].tolist()[0]
        s_pessoa2 = jogos[0].iloc[[x]]['saldo_'+pessoa2].tolist()[0]
        valor = jogos[0].iloc[[x]]['valor2'].tolist()[0]
        s_pessoa1 = s_pessoa1 - valor
        s_pessoa2 = s_pessoa2 - valor        
    elif acao == "tratamento":
        #Não vai funcionar 100%
        tratamento = jogos[0].iloc[[x]]['tratamento'].tolist()[0]
        pessoa1 = jogos[0].iloc[[x]]['pessoa1'].tolist()[0]
        s_pessoa1 = jogos[0].iloc[[x]]['saldo_'+pessoa1].tolist()[0]        
        poluicao = jogos[0].iloc[[x]]['poluicao_mundial'].tolist()[0]
        p = 0
        #Verificar esse if
        if poluicao >= 30:            
            p = poluicao - 20
        if tratamento == 'Lixo':
            s_pessoa1 = -1600-p
        elif tratamento == 'Agua':
            s_pessoa1 = -800-p
        else:
            #tratamento de esgoto
            s_pessoa1 = -2400-p
    elif acao == "nova taxa":
        prox_taxa = int(jogos[0].iloc[[x]]['prox_taxa'].tolist()[0])
        if prox_taxa == 1:
            print('')
        elif prox_taxa == 2:
            print('')
        else: 
            #taxa 3
            print('')
    
    if acao == 'venda' or acao == 'aluguel': # atualiza a produtividade do agricultor conforme o passar da rodada
        if agricultor == 'AT1':
            produtividade[0]+ = produtividade_agricultor
        elif agricultor == 'AT2':
            produtividade[1]+ = produtividade_agricultor
        elif agricultor == 'AT3':
            produtividade[2]+ = produtividade_agricultor
        elif agricultor == 'CD1':
            produtividade[3]+ = produtividade_agricultor
        elif agricultor == 'CD2':
            produtividade[4]+ = produtividade_agricultor
        else:
            produtividade[5]+ = produtividade_agricultor

In [25]:
#df_final = pd.DataFrame([])
#for i in range(0,12):
#    df_final = pd.concat([df_final, rodadas[i]])